<a href="https://colab.research.google.com/github/srilamaiti/fall_2023_210_capstone_deepika_srila/blob/main/final_deliverable/model/train_val_test_file_generation%20/generate_train_val_test_files_8classes_webscraped_and_recorded_videos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
import random
import tqdm
import sys
import pathlib
import itertools
import collections
from pathlib import Path
from sklearn.utils import shuffle
import shutil
import glob
from glob import glob
import urllib.request
import subprocess
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
os.environ['GDRIVE_CONFIG_DIR'] = "/content/drive/MyDrive"
import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

Mounted at /content/drive


In [26]:
LABEL_LIST = ['hello', 'nice', 'meet', 'you', 'thankyou', 'teacher', 'class', 'love']

label_info_dict = {}
for label in LABEL_LIST:
    label_info_dict[label] = {'gdrive_path_1' : os.path.join(os.environ['GDRIVE_CONFIG_DIR'], 'recorded_videos', f'{label}_avi_videos'),
                              'gdrive_path_1_file_count' : len(glob(os.path.join(os.environ['GDRIVE_CONFIG_DIR'], 'recorded_videos', f'{label}_avi_videos') + "/*.avi")),
                              'gdrive_path_2' : os.path.join(os.environ['GDRIVE_CONFIG_DIR'], 'webscraped_videos', f'{label}_avi_videos'),
                              'gdrive_path_2_file_count' : len(glob(os.path.join(os.environ['GDRIVE_CONFIG_DIR'], 'webscraped_videos', f'{label}_avi_videos') + "/*.avi"))
                             }
label_info_dict

{'hello': {'gdrive_path_1': '/content/drive/MyDrive/recorded_videos/hello_avi_videos',
  'gdrive_path_1_file_count': 189,
  'gdrive_path_2': '/content/drive/MyDrive/webscraped_videos/hello_avi_videos',
  'gdrive_path_2_file_count': 90},
 'nice': {'gdrive_path_1': '/content/drive/MyDrive/recorded_videos/nice_avi_videos',
  'gdrive_path_1_file_count': 171,
  'gdrive_path_2': '/content/drive/MyDrive/webscraped_videos/nice_avi_videos',
  'gdrive_path_2_file_count': 90},
 'meet': {'gdrive_path_1': '/content/drive/MyDrive/recorded_videos/meet_avi_videos',
  'gdrive_path_1_file_count': 198,
  'gdrive_path_2': '/content/drive/MyDrive/webscraped_videos/meet_avi_videos',
  'gdrive_path_2_file_count': 90},
 'you': {'gdrive_path_1': '/content/drive/MyDrive/recorded_videos/you_avi_videos',
  'gdrive_path_1_file_count': 180,
  'gdrive_path_2': '/content/drive/MyDrive/webscraped_videos/you_avi_videos',
  'gdrive_path_2_file_count': 72},
 'thankyou': {'gdrive_path_1': '/content/drive/MyDrive/recorded_

In [28]:
SPLITS = (.6, .2, .2)
TOTAL_SAMPLES_PER_LABEL_RECORDED_VIDEOS = 100
TOTAL_SAMPLES_PER_LABEL_WEBSCRAPED_VIDEOS = 60

TRAIN_RECS_PER_LABEL_RECORDED_VIDEOS = int(TOTAL_SAMPLES_PER_LABEL_RECORDED_VIDEOS * SPLITS[0])
VAL_RECS_PER_LABEL_RECORDED_VIDEOS = int(TOTAL_SAMPLES_PER_LABEL_RECORDED_VIDEOS * SPLITS[1])
TEST_RECS_PER_LABEL_RECORDED_VIDEOS = int(TOTAL_SAMPLES_PER_LABEL_RECORDED_VIDEOS * SPLITS[2])

print(f"TRAIN_RECS_PER_LABEL_RECORDED_VIDEOS : {TRAIN_RECS_PER_LABEL_RECORDED_VIDEOS}")
print(f"VAL_RECS_PER_LABEL_RECORDED_VIDEOS : {VAL_RECS_PER_LABEL_RECORDED_VIDEOS}")
print(f"TEST_RECS_PER_LABEL_RECORDED_VIDEOS : {TEST_RECS_PER_LABEL_RECORDED_VIDEOS}")

TRAIN_RECS_PER_LABEL_WEBSCRAPED_VIDEOS = int(TOTAL_SAMPLES_PER_LABEL_WEBSCRAPED_VIDEOS * SPLITS[0])
VAL_RECS_PER_LABEL_WEBSCRAPED_VIDEOS = int(TOTAL_SAMPLES_PER_LABEL_WEBSCRAPED_VIDEOS * SPLITS[1])
TEST_RECS_PER_LABEL_WEBSCRAPED_VIDEOS = int(TOTAL_SAMPLES_PER_LABEL_WEBSCRAPED_VIDEOS * SPLITS[2])

print(f"TRAIN_RECS_PER_LABEL_WEBSCRAPED_VIDEOS : {TRAIN_RECS_PER_LABEL_WEBSCRAPED_VIDEOS}")
print(f"VAL_RECS_PER_LABEL_WEBSCRAPED_VIDEOS : {VAL_RECS_PER_LABEL_WEBSCRAPED_VIDEOS}")
print(f"TEST_RECS_PER_LABEL_WEBSCRAPED_VIDEOS : {TEST_RECS_PER_LABEL_WEBSCRAPED_VIDEOS}")

TRAIN_RECS_PER_LABEL_RECORDED_VIDEOS : 60
VAL_RECS_PER_LABEL_RECORDED_VIDEOS : 20
TEST_RECS_PER_LABEL_RECORDED_VIDEOS : 20
TRAIN_RECS_PER_LABEL_WEBSCRAPED_VIDEOS : 36
VAL_RECS_PER_LABEL_WEBSCRAPED_VIDEOS : 12
TEST_RECS_PER_LABEL_WEBSCRAPED_VIDEOS : 12


In [30]:
TRAIN_FILE_LIST = []
VAL_FILE_LIST = []
TEST_FILE_LIST = []

def create_dir(dir_path):
    os.makedirs(dir_path, exist_ok=True)

def copy_file(file_name, dir_path):
    !cp {file} {dir_path}

def copy_set_of_files(file_list, dir_path):
    #print(len(file_list), dir_path)
    for file in file_list:
        #print(f"Copying {file} to {dir_path}")
        #copy_file(file_name = file, dir_path = dir_path)
        !cp {file} {dir_path}
    print(f"File count in {dir_path} : {len(os.listdir(dir_path))}")
    #print(os.listdir(dir_path))

def remove_files_from_dir(dir_path):
    for file in os.listdir(dir_path):
        qualified_file_name = os.path.join(dir_path, file)
        !rm -rf {qualified_file_name}
    print(f"File count in {dir_path} : {len(os.listdir(dir_path))}")

random.seed(1234)
os.environ['PYTHONHASHSEED'] = '1234'

# Saving files in gdrive
#current_ts = datetime.today().isoformat().replace("-","").replace("T","").replace(":","").replace(".","")
TRAIN_FILE_NAME = os.path.join(os.environ['GDRIVE_CONFIG_DIR'], '8classes_train_val_test_files_recorded_and_webscraped_videos', 'train_files.txt')
VAL_FILE_NAME = os.path.join(os.environ['GDRIVE_CONFIG_DIR'], '8classes_train_val_test_files_recorded_and_webscraped_videos', 'val_files.txt')
TEST_FILE_NAME = os.path.join(os.environ['GDRIVE_CONFIG_DIR'], '8classes_train_val_test_files_recorded_and_webscraped_videos', 'test_files.txt')

train_file_pointer = open(TRAIN_FILE_NAME, 'w')
val_file_pointer = open(VAL_FILE_NAME, 'w')
test_file_pointer = open(TEST_FILE_NAME, 'w')

for label in LABEL_LIST:
    gdrive_path_1 = label_info_dict[label]['gdrive_path_1'] # recorded videos
    gdrive_path_2 = label_info_dict[label]['gdrive_path_2'] # webscraped videos

    # test
    temp_test_files_1 = subprocess.getoutput(f"shuf -zn{TEST_RECS_PER_LABEL_RECORDED_VIDEOS} -e {gdrive_path_1}/*.avi ").split("\n")
    test_files_1 = [e.split(" -> ")[0].replace("'","") for e in temp_test_files_1]
    test_files_1 = test_files_1[0].split("\x00")[:-1]
    temp_test_files_2 = subprocess.getoutput(f"shuf -zn{TEST_RECS_PER_LABEL_WEBSCRAPED_VIDEOS} -e {gdrive_path_2}/*.avi ").split("\n")
    test_files_2 = [e.split(" -> ")[0].replace("'","") for e in temp_test_files_2]
    test_files_2 = test_files_2[0].split("\x00")[:-1]
    test_files = test_files_1 + test_files_2

    set_of_all_files = list(set([os.path.join(gdrive_path_1, file) for file in os.listdir(gdrive_path_1)] + [os.path.join(gdrive_path_2, file) for file in os.listdir(gdrive_path_2)]))

    # train
    temp_train_files_1 = list(set([os.path.join(gdrive_path_1, file) for file in os.listdir(gdrive_path_1)]) - set(test_files_1))
    temp_train_files_1 = sorted(temp_train_files_1, key=lambda x: random.random())
    train_files_1 = random.sample(temp_train_files_1, TRAIN_RECS_PER_LABEL_RECORDED_VIDEOS)
    temp_train_files_2 = list(set([os.path.join(gdrive_path_2, file) for file in os.listdir(gdrive_path_2)]) - set(test_files_2))
    temp_train_files_2 = sorted(temp_train_files_2, key=lambda x: random.random())
    train_files_2 = random.sample(temp_train_files_2, TRAIN_RECS_PER_LABEL_WEBSCRAPED_VIDEOS)
    train_files = train_files_1 + train_files_2

    # val
    temp_val_files_1 = list(set([os.path.join(gdrive_path_1, file) for file in os.listdir(gdrive_path_1)]) - set(test_files_1) - set(train_files_1))
    temp_val_files_1 = sorted(temp_val_files_1, key=lambda x: random.random())
    val_files_1 = random.sample(temp_val_files_1, VAL_RECS_PER_LABEL_RECORDED_VIDEOS)
    temp_val_files_2 = list(set([os.path.join(gdrive_path_2, file) for file in os.listdir(gdrive_path_2)]) - set(test_files_2) - set(train_files_2))
    temp_val_files_2 = sorted(temp_val_files_2, key=lambda x: random.random())
    val_files_2 = random.sample(temp_val_files_2, VAL_RECS_PER_LABEL_WEBSCRAPED_VIDEOS)
    val_files = val_files_1 + val_files_2

    [train_file_pointer.writelines(e + "\n") for e in train_files]
    [val_file_pointer.writelines(e + "\n") for e in val_files]
    [test_file_pointer.writelines(e + "\n") for e in test_files]

# File closing
train_file_pointer.close()
val_file_pointer.close()
test_file_pointer.close()

In [31]:
!cat {TRAIN_FILE_NAME}|wc -l

768


In [32]:
!cat {TEST_FILE_NAME}|wc -l

256


In [33]:
!cat {VAL_FILE_NAME}|wc -l

256


In [34]:
TRAIN_FILE_PATH = os.path.join(os.environ['GDRIVE_CONFIG_DIR'], '8classes_train_val_test_files_recorded_and_webscraped_videos', 'train')
VAL_FILE_PATH = os.path.join(os.environ['GDRIVE_CONFIG_DIR'], '8classes_train_val_test_files_recorded_and_webscraped_videos', 'val')
TEST_FILE_PATH = os.path.join(os.environ['GDRIVE_CONFIG_DIR'], '8classes_train_val_test_files_recorded_and_webscraped_videos', 'test')

for label in LABEL_LIST:
    train_dir_name = os.path.join(TRAIN_FILE_PATH, label)
    create_dir(train_dir_name)

    val_dir_name = os.path.join(VAL_FILE_PATH, label)
    create_dir(val_dir_name)

    test_dir_name = os.path.join(TEST_FILE_PATH, label)
    create_dir(test_dir_name)

In [35]:
def copy_file_to_folder(file_name, dir_path):
    fp = open(file_name, "r")
    for file in fp:
        file = file.strip()
        label = file.split("/")[-2].split("_")[0]
        base_file_name = os.path.basename(file)
        dest_path = os.path.join(dir_path, label)
        shutil.copy(file, dest_path)
        #!ls -ltrSh {dir_path}/{label}/*.avi | wc -l

In [36]:
for label in LABEL_LIST:
    !rm -rf {TRAIN_FILE_PATH}/{label}/*.avi
    !rm -rf {VAL_FILE_PATH}/{label}/*.avi
    !rm -rf {TEST_FILE_PATH}/{label}/*.avi

copy_file_to_folder(file_name = TRAIN_FILE_NAME, dir_path = TRAIN_FILE_PATH)
copy_file_to_folder(file_name = VAL_FILE_NAME, dir_path = VAL_FILE_PATH)
copy_file_to_folder(file_name = TEST_FILE_NAME, dir_path = TEST_FILE_PATH)

for label in LABEL_LIST:
    !ls -ltrSh {TRAIN_FILE_PATH}/{label}/*.avi | wc -l
    !ls -ltrSh {VAL_FILE_PATH}/{label}/*.avi | wc -l
    !ls -ltrSh {TEST_FILE_PATH}/{label}/*.avi | wc -l

96
32
32
96
32
32
96
32
32
96
32
32
96
32
32
96
32
32
96
32
32
96
32
32
